In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the dataset (assuming the DataFrame 'df' is already loaded with 'tweet' and 'label' columns)
df = pd.read_csv("../data/mendeley_data/HateSpeechDatasetBalanced.csv")
tweets = df['Content'].values
labels = df['Label'].values

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2, random_state=42)

# Load the saved BiLSTM model
model_bilstm = load_model('bilstm_model.h5')

# Unfreeze the last 2 layers
for layer in model_bilstm.layers[:-1]:
    layer.trainable = False


In [3]:
# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(tweets)
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=100, padding='post', truncating='post')
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=100, padding='post', truncating='post')


In [5]:
# Compile the model with a smaller learning rate
model_bilstm.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune the BiLSTM model
history_bilstm_fine_tune = model_bilstm.fit(X_train_seq, y_train, epochs=10, batch_size=128, validation_split=0.2)

# Save the fine-tuned BiLSTM model
model_bilstm.save('bilstm_model_fine_tuned_last1_layer.h5')
print('Fine-tuned BiLSTM model saved as bilstm_model_fine_tuned.h5')

Epoch 1/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 637s 174ms/step - accuracy: 0.9100 - loss: 0.2148 - val_accuracy: 0.8638 - val_loss: 0.3209
Epoch 2/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 726s 200ms/step - accuracy: 0.9107 - loss: 0.2128 - val_accuracy: 0.8637 - val_loss: 0.3250
Epoch 3/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 913s 251ms/step - accuracy: 0.9116 - loss: 0.2108 - val_accuracy: 0.8636 - val_loss: 0.3279
Epoch 4/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 848s 231ms/step - accuracy: 0.9115 - loss: 0.2110 - val_accuracy: 0.8637 - val_loss: 0.3300
Epoch 5/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 797s 220ms/step - accuracy: 0.9118 - loss: 0.2092 - val_accuracy: 0.8637 - val_loss: 0.3314
Epoch 6/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 882s 243ms/step - accuracy: 0.9114 - loss: 0.2101 - val_accuracy: 0.8637 - val_loss: 0.3324
Epoch 7/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 907s 250ms/step - accuracy: 0.9109 - loss: 0.2107 - val_accuracy: 0.8637 - val_loss: 0.3331
Epoch 8/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 885s 244ms/step - ac

Fine-tuned BiLSTM model saved as bilstm_model_fine_tuned.h5


In [4]:
tweets = df['Content'].values
labels = df['Label'].values

# TF-IDF Vectorization
# # Split the data
# X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)


In [6]:
vectorizer = TfidfVectorizer(max_features=2000) 
X_tfidf = vectorizer.fit_transform(tweets)


In [7]:
# Split the data
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)


In [10]:
# Load the saved TF-IDF model
model_tfidf = load_model('tfidf_model.h5')
model_tfidf.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 2000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     1,024,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,156,099 (4.41 MB)

 Trainable params: 1,156,097 (4.41 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [11]:
# Unfreeze the last 2 layers
for layer in model_tfidf.layers[:-1]:
    layer.trainable = False

In [12]:
# Compile the model with a smaller learning rate
model_tfidf.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
# Fine-tune the TF-IDF model
history_tfidf_fine_tune = model_tfidf.fit(X_train_tfidf, y_train, epochs=10, batch_size=128, validation_split=0.2)


Epoch 1/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 49s 13ms/step - accuracy: 0.6935 - loss: 0.9133 - val_accuracy: 0.6947 - val_loss: 0.6968
Epoch 2/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 50s 14ms/step - accuracy: 0.6902 - loss: 0.6886 - val_accuracy: 0.6849 - val_loss: 0.5947
Epoch 3/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 46s 12ms/step - accuracy: 0.6811 - loss: 0.6111 - val_accuracy: 0.6816 - val_loss: 0.5832
Epoch 4/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 55s 15ms/step - accuracy: 0.6788 - loss: 0.6013 - val_accuracy: 0.6841 - val_loss: 0.5790
Epoch 5/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 48s 13ms/step - accuracy: 0.6824 - loss: 0.5943 - val_accuracy: 0.6868 - val_loss: 0.5762
Epoch 6/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 46s 13ms/step - accuracy: 0.6838 - loss: 0.5905 - val_accuracy: 0.6899 - val_loss: 0.5740
Epoch 7/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 46s 13ms/step - accuracy: 0.6863 - loss: 0.5862 - val_accuracy: 0.6931 - val_loss: 0.5726
Epoch 8/10
3631/3631 ━━━━━━━━━━━━━━━━━━━━ 46s 13ms/step - accuracy: 0.6898 -

In [14]:
# Save the fine-tuned TF-IDF model
model_tfidf.save('tfidf_model_fine_tuned_last1_layer.h5')
print('Fine-tuned TF-IDF model saved as tfidf_model_fine_tuned.h5')

Fine-tuned TF-IDF model saved as tfidf_model_fine_tuned.h5


In [15]:
model_tfidf.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 2000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │     1,024,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,156,613 (4.41 MB)

 Trainable params: 257 (1.00 KB)

 Non-trainable params: 1,155,840 (4.41 MB)

 Optimizer params: 516 (2.02 KB)